using variosu approaches nearby area 

In [8]:
import pandas as pd
import requests
import time
import googlemaps
import json
import re
from typing import List, Dict, Optional
from dotenv import load_dotenv
import os

df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\3_2.csv')
print(f"✅ Data loaded: {df.shape[0]} locations")
#get a random subset of 3 entries
df = df.sample(n=3, random_state=42).reset_index(drop=True)
df.head()

✅ Data loaded: 14 locations


,Unnamed: 0,filename,record_num,clean_line1,clean_line2,line3,city,zip_code,label,phone,...,places_website,places_street_number,places_street_name,places_city,places_state,places_postal_code,places_country,places_county,places_google_maps_link,address_string
0,23232,TF2016_260_275-0-ocr.csv,27,D Lake Point ( 84074 ) TA Tooele # 060 ( Shell...,801-250-8585 1-80 Exit 99,Z XL,Lake Point,84074,TA Tooele # 060 ( Shell ) Z,801-250-8585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Shell, TA Tooele # 060 ( Shell ) Z, I-80 Exit ..."
1,23262,TF2016_260_275-1-ocr.csv,25,D Salt Lake City ( W ) ( 84104 ) Sapp Bros. - ...,3 801-977-3900 1-215 Exit 21 ( California Ave E ),24 D XL 24,Salt Lake City,84104,Sapp Bros. - Salt Lake City ( Sinclair ),801-977-3900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sinclair, Sapp Bros. - Salt Lake City ( Sincla..."
2,23550,TF2016_004_019-5-ocr.csv,24,G Gila Bend ( 85337 ) Holt Interstate Services...,928-683-2449 1-8 Exit 119,XL,Gila Bend,85337,Holt Interstate Services ( Shell ),928-683-2449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Shell, Holt Interstate Services ( Shell ), I-8..."
3,23982,TF2016_144_161-7-ocr.csv,10,D Fallon ( 89406 ) Harmon Junction,775-423-3888 US 50 & Harmon Rd ( 5 mi E of,) <U+25A1> S <U+2610>,Fallon,89406,Harmon Junction,775-423-3888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Harmon Junction, US 50 and Harmon Rd and 5 mi ..."
4,24024,TF2016_144_161-7-ocr.csv,37,Sparks ( 89431 ) TA Sparks # 172 ( Valero ),775-359-0550 1-80 Exit 19 ( McCarran Blvd ),24 FORS XL,Sparks,89431,TA Sparks # 172 ( Valero ),775-359-0550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Valero, TA Sparks # 172 ( Valero ), I-80 Exit ..."


In [9]:
# Data exploration and preprocessing
print("📊 DataFrame structure:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Extract chain information from label
def extract_chain_from_label(label):
    """Extract fuel chain from the label"""
    if pd.isna(label):
        return 'Unknown'
    
    # Common fuel chains to look for
    chains = ['Shell', 'Chevron', 'Mobil', 'Exxon', 'BP', 'Texaco', 'Sinclair', 
             'Valero', '76', 'Citgo', 'Marathon', 'Speedway', 'Phillips 66',
             'Conoco', 'Sunoco', 'Arco', 'Hess', 'Murphy', "Casey's"]
    
    label_upper = label.upper()
    for chain in chains:
        if chain.upper() in label_upper:
            return chain
    return 'Unknown'

# Add chain column if it doesn't exist
if 'chain' not in df.columns:
    df['chain'] = df['label'].apply(extract_chain_from_label)
    print("✅ Chain information extracted from labels")

print("\n🏪 Chain distribution:")
print(df['chain'].value_counts())

📊 DataFrame structure:
Shape: (14, 46)
Columns: ['Unnamed: 0', 'filename', 'record_num', 'clean_line1', 'clean_line2', 'line3', 'city', 'zip_code', 'label', 'phone', 'address', 'parking', 'gray_parking', 'identifier', 'year', 'panel', 'major_city', 'state', 'chain', 'p_identifier', 'identifier2', 'identifier3', 'places_search_query', 'places_place_id', 'places_name', 'places_formatted_address', 'places_vicinity', 'places_latitude', 'places_longitude', 'places_rating', 'places_types', 'places_business_status', 'places_search_method', 'places_status', 'places_formatted_phone', 'places_international_phone', 'places_website', 'places_street_number', 'places_street_name', 'places_city', 'places_state', 'places_postal_code', 'places_country', 'places_county', 'places_google_maps_link', 'address_string']

🏪 Chain distribution:
chain
Shell       4
Chevron     3
Valero      2
Sinclair    1
Mobil       1
Name: count, dtype: int64


In [10]:
import pandas as pd
import requests
import time
from urllib.parse import quote
import googlemaps
import json
from typing import List, Dict, Optional
from dotenv import load_dotenv
import os

# API Setup and Google Places Class
load_dotenv()
PLACES_API_KEY = os.getenv("PLACES_API_KEY")
API_KEY=PLACES_API_KEY
if not PLACES_API_KEY:
    print("❌ API_KEY not found!")
    print("\n🔧 Setup Instructions:")
    print("1. Create a .env file in this directory")
    print("2. Add: PLACES_API_KEY=your_actual_api_key_here")
    print("3. Get API key from: https://console.cloud.google.com/")
    print("4. Enable Places API and Geocoding API")
    raise ValueError("API_KEY not found. Please follow setup instructions above.")

print("✅ API Key loaded successfully!")

class GooglePlacesNearbySearch:
    def __init__(self, api_key: str):
        self.gmaps = googlemaps.Client(key=api_key)
    
    def get_coordinates_from_address(self, city: str, state: str, zip_code: str) -> Optional[tuple]:
        """Get latitude and longitude from city, state, zip code"""
        try:
            address = f"{city}, {state} {zip_code}"
            geocode_result = self.gmaps.geocode(address)
            
            if geocode_result:
                location = geocode_result[0]['geometry']['location']
                return (location['lat'], location['lng'])
            return None
        except Exception as e:
            print(f"Error geocoding {city}, {state}: {e}")
            return None
    
    def nearby_search(self, latitude: float, longitude: float, 
                     keyword: str = None, place_type: str = "gas_station", 
                     radius: int = 5000) -> List[Dict]:
        """Search for nearby places using Google Places API"""
        try:
            params = {
                'location': (latitude, longitude),
                'radius': radius,
                'type': place_type
            }
            
            if keyword:
                params['keyword'] = keyword
            
            places_result = self.gmaps.places_nearby(**params)
            return places_result.get('results', [])
            
        except Exception as e:
            print(f"Error in nearby search: {e}")
            return []
    
    def search_similar_places(self, row_data: Dict) -> List[Dict]:
        """Search for places similar to the known location using three strategies"""
        coords = self.get_coordinates_from_address(
            row_data['city'], row_data['state'], str(row_data['zip_code'])
        )
        
        if not coords:
            return []
        
        lat, lng = coords
        search_results = []
        
        # Strategy 1: Search by chain name (10km radius)
        if row_data.get('chain') and row_data['chain'] != 'Unknown':
            chain_results = self.nearby_search(
                lat, lng, keyword=row_data['chain'], 
                place_type="gas_station", radius=10000
            )
            search_results.extend(chain_results)
        
        # Strategy 2: General gas stations (5km radius)
        gas_station_results = self.nearby_search(
            lat, lng, place_type="gas_station", radius=5000
        )
        search_results.extend(gas_station_results)
        
        # Strategy 3: Business name search (15km radius)
        if row_data.get('label'):
            business_name = row_data['label'].split('(')[0].strip()
            if len(business_name) > 3:
                name_results = self.nearby_search(
                    lat, lng, keyword=business_name,
                    place_type="establishment", radius=15000
                )
                search_results.extend(name_results)
        
        # Remove duplicates based on place_id
        unique_results = []
        seen_place_ids = set()
        
        for result in search_results:
            place_id = result.get('place_id')
            if place_id and place_id not in seen_place_ids:
                seen_place_ids.add(place_id)
                unique_results.append(result)
        
        return unique_results

# Initialize the client
try:
    places_client = GooglePlacesNearbySearch(PLACES_API_KEY)
    print("🌐 Google Places client initialized successfully")
    
    # Also create a standalone gmaps client for direct access
    gmaps = googlemaps.Client(key=PLACES_API_KEY)
    print("✅ Standalone gmaps client also initialized")
    
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")

✅ API Key loaded successfully!
🌐 Google Places client initialized successfully
✅ Standalone gmaps client also initialized


In [12]:
def process_dataframe_with_nearby_search(df, api_key, max_results_per_location=10, delay_between_requests=0.1):
    """
    Process the entire DataFrame and find nearby places for each location
    """
    if not api_key or api_key == "YOUR_GOOGLE_MAPS_API_KEY":
        print("⚠️  Please set your Google Maps API key first!")
        return None
    
    places_client = GooglePlacesNearbySearch(api_key)
    
    # Store results
    all_results = []
    
    print(f"🚀 Processing {len(df)} locations...")
    
    for idx, row in df.iterrows():
        print(f"\n[{idx+1}/{len(df)}] Processing: {row['label']} in {row['city']}, {row['state']}")
        
        # Convert row to dictionary
        row_dict = {
            'city': row['city'],
            'state': row['state'],
            'zip_code': row['zip_code'],
            'chain': row['chain'],
            'label': row['label']
        }
        
        try:
            # Get nearby places
            nearby_places = places_client.search_similar_places(row_dict)
            
            # Limit results per location
            nearby_places = nearby_places[:max_results_per_location]
            
            print(f"   Found {len(nearby_places)} nearby places")
            
            # Add original row data to each result
            for place in nearby_places:
                result_record = {
                    'original_index': idx,
                    'original_city': row['city'],
                    'original_state': row['state'],
                    'original_zip': row['zip_code'],
                    'original_chain': row['chain'],
                    'original_label': row['label'],
                    'original_phone': row.get('phone', ''),
                    
                    # Nearby place data
                    'nearby_place_id': place.get('place_id'),
                    'nearby_name': place.get('name'),
                    'nearby_address': place.get('vicinity'),
                    'nearby_rating': place.get('rating'),
                    'nearby_types': ', '.join(place.get('types', [])),
                    'nearby_price_level': place.get('price_level'),
                    'nearby_lat': place.get('geometry', {}).get('location', {}).get('lat'),
                    'nearby_lng': place.get('geometry', {}).get('location', {}).get('lng'),
                    'nearby_open_now': place.get('opening_hours', {}).get('open_now')
                }
                all_results.append(result_record)
            
            # Rate limiting to respect API limits
            time.sleep(delay_between_requests)
            
        except Exception as e:
            print(f"   ❌ Error processing row {idx}: {e}")
            continue
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(all_results)
    
    print(f"\n✅ Processing complete! Found {len(results_df)} total nearby places.")
    
    return results_df

# Batch Processing Functions
def process_all_locations(max_results_per_location=5, delay_between_requests=0.2):
    """Process entire DataFrame and find nearby places for each location"""
    all_results = []
    
    print(f"🚀 Processing {len(df)} locations...")
    
    for idx, row in df.iterrows():
        print(f"\n[{idx+1}/{len(df)}] Processing: {row['label']} in {row['city']}, {row['state']}")
        
        row_dict = {
            'city': row['city'],
            'state': row['state'],
            'zip_code': row['zip_code'],
            'chain': row['chain'],
            'label': row['label']
        }
        
        try:
            nearby_places = places_client.search_similar_places(row_dict)
            nearby_places = nearby_places[:max_results_per_location]
            
            print(f"   ✅ Found {len(nearby_places)} nearby places")
            
            # Create result records
            for place in nearby_places:
                result_record = {
                    'original_index': idx,
                    'original_city': row['city'],
                    'original_state': row['state'],
                    'original_zip': row['zip_code'],
                    'original_chain': row['chain'],
                    'original_label': row['label'],
                    'original_phone': row.get('phone', ''),
                    
                    'nearby_place_id': place.get('place_id'),
                    'nearby_name': place.get('name'),
                    'nearby_address': place.get('vicinity'),
                    'nearby_rating': place.get('rating'),
                    'nearby_types': ', '.join(place.get('types', [])),
                    'nearby_price_level': place.get('price_level'),
                    'nearby_lat': place.get('geometry', {}).get('location', {}).get('lat'),
                    'nearby_lng': place.get('geometry', {}).get('location', {}).get('lng'),
                    'nearby_open_now': place.get('opening_hours', {}).get('open_now')
                }
                all_results.append(result_record)
            
            time.sleep(delay_between_requests)
            
        except Exception as e:
            print(f"   ❌ Error processing row {idx}: {e}")
            continue
    
    results_df = pd.DataFrame(all_results)
    print(f"\n✅ Processing complete! Found {len(results_df)} total nearby places.")
    return results_df

# Function to analyze the results
def analyze_nearby_results(results_df):
    """Analyze the nearby search results"""
    if results_df is None or len(results_df) == 0:
        print("No results to analyze")
        return
    
    print("📊 Analysis of Nearby Search Results:")
    print(f"Total nearby places found: {len(results_df)}")
    print(f"Average places per original location: {len(results_df) / results_df['original_index'].nunique():.1f}")
    
    print("\n🏪 Most common nearby place types:")
    # Flatten and count all types
    all_types = []
    for types_str in results_df['nearby_types'].dropna():
        all_types.extend([t.strip() for t in types_str.split(',') if t.strip()])
    
    type_counts = pd.Series(all_types).value_counts().head(10)
    for place_type, count in type_counts.items():
        print(f"   {place_type}: {count}")
    
    print("\n⭐ Rating distribution:")
    rating_stats = results_df['nearby_rating'].describe()
    print(f"   Average rating: {rating_stats['mean']:.2f}")
    print(f"   Rating range: {rating_stats['min']:.1f} - {rating_stats['max']:.1f}")
    
    return results_df

def analyze_results(results_df):
    """Analyze and summarize the nearby search results"""
    if results_df is None or len(results_df) == 0:
        print("No results to analyze")
        return
    
    print("\n📊 Analysis of Results:")
    print(f"📍 Total nearby places found: {len(results_df)}")
    print(f"🎯 Average places per location: {len(results_df) / results_df['original_index'].nunique():.1f}")
    
    # Most common place types
    print("\n🏪 Most common place types:")
    all_types = []
    for types_str in results_df['nearby_types'].dropna():
        all_types.extend([t.strip() for t in types_str.split(',') if t.strip()])
    
    type_counts = pd.Series(all_types).value_counts().head(8)
    for place_type, count in type_counts.items():
        print(f"   {place_type}: {count}")
    
    # Rating statistics
    print("\n⭐ Rating statistics:")
    rating_stats = results_df['nearby_rating'].describe()
    print(f"   Average rating: {rating_stats['mean']:.2f}")
    print(f"   Rating range: {rating_stats['min']:.1f} - {rating_stats['max']:.1f}")
    
    return results_df

print("🔧 Batch processing functions ready!")
print("📋 Use process_all_locations() to process all your locations.")

# Test API connection with a simple geocoding request
print("🧪 Testing API connection...")

try:
    # Use the places_client.gmaps instance that was already created
    test_geocode = places_client.gmaps.geocode("New York, NY")
    if test_geocode:
        location = test_geocode[0]['geometry']['location']
        print(f"✅ API connection successful!")
        print(f"📍 Test geocoding result: New York, NY = {location['lat']:.4f}, {location['lng']:.4f}")
        
        # Test nearby search
        test_nearby = places_client.gmaps.places_nearby(
            location=(location['lat'], location['lng']),
            radius=5000,
            type='gas_station'
        )
        print(f"🏪 Found {len(test_nearby.get('results', []))} gas stations near NYC")
        print("🎉 All API functions working correctly!")
        
    else:
        print("❌ Geocoding test failed")
        
except Exception as e:
    print(f"❌ API connection failed: {e}")
    print("Please check:")
    print("1. Your .env file exists and contains PLACES_API_KEY=your_key_here")
    print("2. Places API and Geocoding API are enabled in Google Cloud Console")
    print("3. Your API key has the correct permissions")

🔧 Batch processing functions ready!
📋 Use process_all_locations() to process all your locations.


In [13]:
# Test API connection with a simple geocoding request
print("🧪 Testing API connection...")

try:
    # Use the places_client.gmaps instance that was already created
    test_geocode = places_client.gmaps.geocode("New York, NY")
    if test_geocode:
        location = test_geocode[0]['geometry']['location']
        print(f"✅ API connection successful!")
        print(f"📍 Test geocoding result: New York, NY = {location['lat']:.4f}, {location['lng']:.4f}")
        
        # Test nearby search
        test_nearby = places_client.gmaps.places_nearby(
            location=(location['lat'], location['lng']),
            radius=5000,
            type='gas_station'
        )
        print(f"🏪 Found {len(test_nearby.get('results', []))} gas stations near NYC")
        print("🎉 All API functions working correctly!")
        
    else:
        print("❌ Geocoding test failed")
        
except Exception as e:
    print(f"❌ API connection failed: {e}")
    print("Please check:")
    print("1. Your .env file exists and contains PLACES_API_KEY=your_key_here")
    print("2. Places API and Geocoding API are enabled in Google Cloud Console")
    print("3. Your API key has the correct permissions")

# Process All Locations and Save Results
print("🏪 Processing all locations in your DataFrame...")
print("=" * 60)

# Process all locations (limit to 5 results per location to manage costs)
results_df = process_all_locations(
    max_results_per_location=5,
    delay_between_requests=0.2
)

if results_df is not None and len(results_df) > 0:
    # Analyze results
    print("\n" + "=" * 60)
    analyze_results(results_df)
    
    # Save to CSV
    output_file = r'C:\Users\clint\Desktop\Geocoding_Task\3_2_2.csv'
    results_df.to_csv(output_file, index=False)
    print(f"\n💾 Results saved to: {output_file}")
    
    # Show sample results
    print(f"\n📋 Sample results (first 5 rows):")
    display(results_df.head())
    
    print(f"\n🎉 Process complete! {len(results_df)} nearby places found and saved.")
else:
    print("❌ No results to process")

🧪 Testing API connection...
✅ API connection successful!
📍 Test geocoding result: New York, NY = 40.7128, -74.0060
🏪 Found 20 gas stations near NYC
🎉 All API functions working correctly!
🎯 Demo has already been run above.
🔄 To run demo again with a different location, use: demo_search(row_index)
📊 Available locations: 0 to 13


In [15]:
# 🚀 Let's run the demo with your actual data!
print("🎯 Running Google Places API demo with your data...")
print("=" * 60)

# Run the demo function with your loaded data
print("🔍 Running demo search with first location...")
demo_results = demo_search(0)  # Demo with first location
print(f"✅ Demo completed! Found {len(demo_results)} places near first location.")

# Process All Locations and Save Results
print("\n🏪 Processing all locations in your DataFrame...")
print("=" * 60)

# Process all locations (limit to 5 results per location to manage costs)
results_df = process_all_locations(
    max_results_per_location=5,
    delay_between_requests=0.2
)

if results_df is not None and len(results_df) > 0:
    # Analyze results
    print("\n" + "=" * 60)
    analyze_results(results_df)
    
    # Save to CSV
    output_file = r'C:\Users\clint\Desktop\Geocoding_Task\3_2_2.csv'
    results_df.to_csv(output_file, index=False)
    print(f"\n💾 Results saved to: {output_file}")
    
    # Show sample results
    print(f"\n📋 Sample results (first 5 rows):")
    display(results_df.head())
    
    print(f"\n🎉 Process complete! {len(results_df)} nearby places found and saved.")
else:
    print("❌ No results to process")

🎯 Running Google Places API demo with your data...
🔍 Running demo search with first location...
🔍 Demo: Searching near TA Tooele # 060 ( Shell ) Z
📍 Location: Lake Point, UT 84074
🏪 Chain: Shell

📍 Found 10 nearby places:

1. Shell
   📍 8836 Clinton Landing Rd, Tooele
   ⭐ Rating: 2.5
   🏷️ Types: gas_station, convenience_store, restaurant
   🌐 Coords: 40.689844, -112.263411

2. Sinclair Gas Station
   📍 430 Bates Canyon Rd, Stansbury Park
   ⭐ Rating: 4.3
   🏷️ Types: gas_station, convenience_store, food
   🌐 Coords: 40.623386, -112.294113

3. Flying J Travel Center
   📍 1605 Saddleback Boulevard, Lake Point
   ⭐ Rating: 4
   🏷️ Types: convenience_store, gas_station, food
   🌐 Coords: 40.686146, -112.266454

4. Sinclair Gas Station
   📍 8764 Utah 36, Lake Point
   ⭐ Rating: 3.1
   🏷️ Types: gas_station, convenience_store, food
   🌐 Coords: 40.688407, -112.265637

5. Chevron
   📍 8793 N Hwy, 8793 Utah 36, Tooele
   ⭐ Rating: 4.1
   🏷️ Types: gas_station, point_of_interest, establishmen

,original_index,original_city,original_state,original_zip,original_chain,original_label,original_phone,nearby_place_id,nearby_name,nearby_address,nearby_rating,nearby_types,nearby_price_level,nearby_lat,nearby_lng,nearby_open_now
0,0,Lake Point,UT,84074,Shell,TA Tooele # 060 ( Shell ) Z,801-250-8585,ChIJycJBJUjAUocRV3VEUtjkDjg,Shell,"8836 Clinton Landing Rd, Tooele",2.5,"gas_station, convenience_store, restaurant, fo...",2.0,40.689844,-112.263411,True
1,0,Lake Point,UT,84074,Shell,TA Tooele # 060 ( Shell ) Z,801-250-8585,ChIJDUomjwW_UocRiTh3672Et8o,Sinclair Gas Station,"430 Bates Canyon Rd, Stansbury Park",4.3,"gas_station, convenience_store, food, point_of...",NaN,40.623386,-112.294113,True
2,0,Lake Point,UT,84074,Shell,TA Tooele # 060 ( Shell ) Z,801-250-8585,ChIJI2JoqfC8UocRG1i941oflRE,Flying J Travel Center,"1605 Saddleback Boulevard, Lake Point",4.0,"gas_station, convenience_store, food, store, p...",NaN,40.686146,-112.266454,True
3,0,Lake Point,UT,84074,Shell,TA Tooele # 060 ( Shell ) Z,801-250-8585,ChIJFVfPjkjAUocRMGLvX59zWyM,Sinclair Gas Station,"8764 Utah 36, Lake Point",3.1,"gas_station, convenience_store, food, store, p...",NaN,40.688407,-112.265637,False
4,0,Lake Point,UT,84074,Shell,TA Tooele # 060 ( Shell ) Z,801-250-8585,ChIJS1px70jAUocRxJqama2aa2I,Chevron,"8793 N Hwy, 8793 Utah 36, Tooele",4.1,"gas_station, point_of_interest, establishment",NaN,40.689152,-112.265974,True



🎉 Process complete! 60 nearby places found and saved.


In [24]:
results_df[["original_label","nearby_address","nearby_name"]]

,original_label,nearby_address,nearby_name
0,TA Tooele # 060 ( Shell ) Z,"8836 Clinton Landing Rd, Tooele",Shell
1,TA Tooele # 060 ( Shell ) Z,"430 Bates Canyon Rd, Stansbury Park",Sinclair Gas Station
2,TA Tooele # 060 ( Shell ) Z,"1605 Saddleback Boulevard, Lake Point",Flying J Travel Center
3,TA Tooele # 060 ( Shell ) Z,"8764 Utah 36, Lake Point",Sinclair Gas Station
4,TA Tooele # 060 ( Shell ) Z,"8793 N Hwy, 8793 Utah 36, Tooele",Chevron
5,Sapp Bros. - Salt Lake City ( Sinclair ),"109 S 300 E, Salt Lake City",Sinclair Gas Station
6,Sapp Bros. - Salt Lake City ( Sinclair ),"210 S 700 E, Salt Lake City",Sinclair Gas Station
7,Sapp Bros. - Salt Lake City ( Sinclair ),"2690 S 700 E, Salt Lake City",Sinclair Gas Station
8,Sapp Bros. - Salt Lake City ( Sinclair ),"3333 W 500 S, Salt Lake City",Sinclair Gas Station
9,Sapp Bros. - Salt Lake City ( Sinclair ),"105 W 600 S, Salt Lake City",Sinclair Gas Station


In [ ]:
df

,Unnamed: 0,filename,record_num,clean_line1,clean_line2,line3,city,zip_code,label,phone,...,places_website,places_street_number,places_street_name,places_city,places_state,places_postal_code,places_country,places_county,places_google_maps_link,address_string
0,23232,TF2016_260_275-0-ocr.csv,27,D Lake Point ( 84074 ) TA Tooele # 060 ( Shell...,801-250-8585 1-80 Exit 99,Z XL,Lake Point,84074,TA Tooele # 060 ( Shell ) Z,801-250-8585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Shell, TA Tooele # 060 ( Shell ) Z, I-80 Exit ..."
1,23262,TF2016_260_275-1-ocr.csv,25,D Salt Lake City ( W ) ( 84104 ) Sapp Bros. - ...,3 801-977-3900 1-215 Exit 21 ( California Ave E ),24 D XL 24,Salt Lake City,84104,Sapp Bros. - Salt Lake City ( Sinclair ),801-977-3900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sinclair, Sapp Bros. - Salt Lake City ( Sincla..."
2,23550,TF2016_004_019-5-ocr.csv,24,G Gila Bend ( 85337 ) Holt Interstate Services...,928-683-2449 1-8 Exit 119,XL,Gila Bend,85337,Holt Interstate Services ( Shell ),928-683-2449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Shell, Holt Interstate Services ( Shell ), I-8..."
3,23982,TF2016_144_161-7-ocr.csv,10,D Fallon ( 89406 ) Harmon Junction,775-423-3888 US 50 & Harmon Rd ( 5 mi E of,) <U+25A1> S <U+2610>,Fallon,89406,Harmon Junction,775-423-3888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Harmon Junction, US 50 and Harmon Rd and 5 mi ..."
4,24024,TF2016_144_161-7-ocr.csv,37,Sparks ( 89431 ) TA Sparks # 172 ( Valero ),775-359-0550 1-80 Exit 19 ( McCarran Blvd ),24 FORS XL,Sparks,89431,TA Sparks # 172 ( Valero ),775-359-0550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Valero, TA Sparks # 172 ( Valero ), I-80 Exit ..."
5,24036,TF2016_144_161-7-ocr.csv,31,B Paradise Hill ( 89445 ) Santa Rosa Station (...,775-623-5547 US 95 & NV 290 ( 23 mi N of Winne...,) S <U+25A1> <U+2610>,Paradise Hill,89445,Santa Rosa Station ( Chevron ),775-623-5547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Chevron, Santa Rosa Station ( Chevron ), US 95..."
6,24168,TF2016_020_035-4-ocr.csv,7,"Ontario ( 91762 ) Petro Ontario # 026 N , San ...",909-390-7800 1-10 Exit 57 ( Milliken Ave ) PETRO,) 24 XL 24 <U+2610>,Ontario,91762,"Petro Ontario # 026 N , San Leandro S )",909-390-7800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Petro Ontario # 026 N , San Leandro S ), I-10 ..."
7,24342,TF2016_020_035-2-ocr.csv,21,G Buttonwillow ( 93206 ) TA Buttonwillow # 160...,4 661-764-5266 1-5 Exit 257 ( CA 58 E ) A,A XL,Buttonwillow,93206,TA Buttonwillow # 160 ( Mobil ),661-764-5266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mobil, TA Buttonwillow # 160 ( Mobil ), I-5 Ex..."
8,24480,TF2016_020_035-3-ocr.csv,54,Mojave ( 93501 ) US Hendy Oil ( 76 ),661-824-4675 CA 14-58 ( W jct ),<U+2610> <U+2610> <U+25A1>,Mojave,93501,US Hendy Oil ( 76 ),661-824-4675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"US Hendy Oil ( 76 ), CA 14-58 and W jct, Mojav..."
9,24534,TF2016_020_035-3-ocr.csv,3,F Fowler ( 93625 ) Buford Star Mart ( # 5 N ( ...,3 559-834-3634 CA 99 Exit 121 ( 24,24 XL <U+25A1>,Fowler,93625,Buford Star Mart ( # 5 N ( Shell to 16111 ) Va...,559-834-3634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Shell, Buford Star Mart ( # 5 N ( Shell to 161..."


: 